# LCA calculations

This notebook is part of supporting information for "Matrix-based Methods for Regionalized Life Cycle Assessment" by Chris Mutel and Stefanie Hellweg, submitted to ES&T.

The most recent version of these notebooks can be found at https://github.com/cmutel/regionalized-lca-examples.

It will not run without the following:

* bw2data, version >= 3.4.2
* bw2calc, version >= 1.7
* bw2regional, version >= 0.5.1
* rower, version >= 0.1
* bw2_lcimpact, version >= 0.2

In [ ]:
def test_installed_software():
    import bw2data
    import bw2calc
    import bw2regional
    import rower
    import bw2_lcimpact

    assert bw2data.__version__ >= (3, 4, 2)
    assert bw2calc.__version__ >= (1, 7)
    assert bw2regional.__version__ >= (0, 5, 1)
    assert rower >= (0, 1)
    assert bw2_lcimpact.__version__ >= (0, 2)
    
test_installed_software()

This notebook just does calculations - not much to explain

In [ ]:
import brightway2 as bw
import bw2regional as bwr
import pandas as pd
import os

In [ ]:
bw.projects.set_current("computational methods paper")

In [ ]:
mix_db = bw.Database("production mixes")

In [ ]:
results = []

for ds in mix_db:
    lca = bw.LCA(
        {ds: 1}, 
        method=('irrigation water', 'surface', 'site-generic'), 
    )
    lca.lci()
    lca.lcia()
    results.append((ds['name'], lca.score))
    
pd.DataFrame(sorted(results), columns=["Crop", "Score"])    

In [ ]:
results = []

for ds in mix_db:
    lca = bwr.OneSpatialScaleLCA(
        {ds: 1}, 
        method=('irrigation water', 'surface', 'state scale'), 
    )
    lca.lci()
    lca.lcia()
    results.append((ds['name'], lca.score))
    
pd.DataFrame(sorted(results), columns=["Crop", "Score"])    

In [ ]:
results = []

for ds in mix_db:
    lca = bwr.TwoSpatialScalesLCA(
        {ds: 1}, 
        method=('irrigation water', 'surface', 'grid scale'), 
    )
    lca.lci()
    lca.lcia()
    results.append((ds['name'], lca.score))
    
pd.DataFrame(sorted(results), columns=["Crop", "Score"])    

In [ ]:
results = []

for ds in mix_db:
    lca = bwr.TwoSpatialScalesWithGenericLoadingLCA(
        {ds: 1}, 
        method=('irrigation water', 'surface', 'grid scale'), 
        loading='irrigation water surface withdrawals'
    )
    lca.lci()
    lca.lcia()
    results.append((ds['name'], lca.score))
    
pd.DataFrame(sorted(results), columns=["Crop", "Score"])    

Extension table is specific to cotton, so don't calculate for other crops

In [ ]:
lca = bwr.ExtensionTablesLCA(
                        {('production mixes', 'Cotton'): 1}, 
                        method=('irrigation water', 'surface', 'grid scale'),
                        xtable="blue water consumption"
                        )
lca.lci()
lca.lcia()
lca.score

## Export some results as GeoJSON

In [ ]:
def export_results_as_geojson(filepath, vector, mapping, geocollection, plot_zeros=False):
    import fiona
    import bw2regional as bwr
    import uuid
    import numpy as np
    
    gc = bwr.geocollections[geocollection]
    total = vector.sum()
    vector = np.squeeze(np.asarray(vector))
    
    mapping = {k[1]: v for k, v in mapping.items() if k[0] == geocollection}
    assert mapping

    with fiona.open(gc['filepath'], 'r') as source:
        schema = {
            'geometry': source.schema['geometry'],
            'properties': {
                'id': source.schema['properties'][gc['field']],
                'absolute': 'float',
                'relative': 'float'
            }
        }
               
        with fiona.open(filepath, 'w', crs=source.crs, driver='GeoJSON', schema=schema) as sink:
            for f in source:
                score = vector[mapping[f['properties'][gc['field']]]]
                if score or plot_zeros:
                    f['properties'] = {
                        gc['field']: f['properties'][gc['field']],
                        'absolute': score,
                        'relative': score / total
                    }
                    sink.write(f)

In [ ]:
lca = bwr.TwoSpatialScalesWithGenericLoadingLCA(
    {('production mixes', 'Cotton'): 1}, 
    method=('irrigation water', 'surface', 'grid scale'), 
    loading='irrigation water surface withdrawals'
)
lca.lci()
lca.lcia()
lca.fix_spatial_dictionaries()

export_results_as_geojson(
    os.path.join("output", "loading_results.geojson"),
    lca.results_ia_spatial_scale().sum(axis=0),
    lca.ia_spatial_dict,
    'water cfs'
)